In [1]:
pip install pymysql 

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql, os, json

file = "sample_data_for_assignment.json"
json_data=open(file,encoding="utf8").read()
json_obj = json.loads(json_data)


# do validation and checks before insert
def validate_string(val):
    if val != None:
        if type(val) is int:
            #for x in val:
            #   print(x)
            return str(val).encode('utf-8')
        else:
            return val


# connect to MySQL
con = pymysql.connect(host = 'localhost',user = 'root',passwd = 'Dhanya2001*',db = 'jsonexample')
cursor = con.cursor()

cursor.execute("DROP TABLE IF EXISTS json_to_sql_table")
cursor.execute("CREATE TABLE json_to_sql_table(name varchar(50),phone varchar(50),email varchar(50),address varchar(50),region varchar(50),country varchar(50),list varchar(50),postalZip varchar(50),currency varchar(50))")
# parse json data to SQL insert
for item in json_obj["data"]:
    for i in range(9):
        validate_string(item[i])
    cursor.execute("INSERT INTO json_to_sql_table VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)", (item[0],item[1],item[2],item[3],item[4],item[5],item[6],item[7],item[8]))
con.commit()
con.close()


 1. unload the data from MySQL to pandas dataframe

In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [84]:
import mysql.connector as connection
import pandas as pd
try:
    mydb = connection.connect(host="localhost", database = 'jsonexample',user="root", passwd="Dhanya2001*",use_pure=True)
    query = "Select * from json_to_sql_table;"
    result_dataFrame = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

C:\Users\SVsre\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [85]:
result_dataFrame.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@protonmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@hotmail.ca,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@yahoo.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@protonmail.edu,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@hotmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


 2. write a program to display all data under their respective column name as a pandas dataframe

In [86]:
# Using DataFrame.loc[] create new DataFrame by specific column.
val = list(result_dataFrame.columns[1:])
df2=result_dataFrame.loc[:, result_dataFrame.columns.drop(val)]
df2


,name
0,Winifred Branch
1,Brielle Davidson
2,Summer Mcfadden
3,Geoffrey Long
4,Barry Conner
...,...
495,Adrian Rodriquez
496,Sonia Lamb
497,Kylee Silva
498,Jeanette Todd


3. write a program to change all email address to have the format -
                    	abc@gmail.com (note the ending should be gmail.com)

In [87]:
for i in range(len(result_dataFrame['email'])):
    item = result_dataFrame['email'][i]
    splited = item.split("@")
    result = str(splited[0]) + "@gmail.com"
    result_dataFrame['email'][i] = result
  

In [88]:
result_dataFrame.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


4. The values for "postalZip" doesn't have same type of data (some are string, some are int, some are alpha numeric). Write a program to convert all values to int. In case of alpha numeric remove the letters and keep only the numbers and convert it to int.

In [58]:
for i in range(len(result_dataFrame['postalZip'])):
    item = result_dataFrame['postalZip'][i]
    new_str = ''.join(char for char in item if char.isnumeric())
    result = int(new_str)
    result_dataFrame['postalZip'][i] = result

In [60]:
pd.to_numeric(result_dataFrame['postalZip'])

0      66242403
1         18317
2          8581
3         28826
4        335049
         ...   
495       49343
496      393741
497       70801
498        3765
499      651193
Name: postalZip, Length: 500, dtype: int64

5. write a separate function which take input data as values under the column name "phone" and processes it to return equivalent ASCII char of the value taken 2 at a time from left to right. Considering the value should be in between 65 to 99. If two-digit number is less than 65 replace it with O (O for Orange)

In [89]:
def phonefun(item):
    new_str = ''.join(char for char in item if char.isnumeric())
    result = new_str
    results = ""
    for i in range(0, len(result), 2):
        a = result[i:i+2]
        #print(a)
        if len(a) == 1 :
            continue
        if int(a)>=65 and int(a)<=99:
            results = results + chr(int(a))
        else:
            results = results + 'O'
    return results
#phonefun("1-811-920-9732")    
    

In [92]:
for i in range(len(result_dataFrame['phone'])):
    item = result_dataFrame['phone'][i]
    r = phonefun(item)
    #print(r)
    result_dataFrame['phone'][i] = str(r)

OOOOc
OOOOO
OO^^B
OOKOb
OOCOH
PJOCV
OOOOO
QAOOE
OO\OI
OOOOO
OOIOO
OObOO
OOOHO
OOMOG
OOVOO
WOOOO
OROOO
OIOOO
bCWCO
OIXOO
OOOOO
OWBON
OVV^O
OOO`O
OOOCO
O`NKK
OOOOO
OOGTO
LOOTL
OOOLO
GIOOO
OONOO
OOOOO
O^OOO
OGOOO
OYOOO
OCDT^
OOOGL
OOOOO
OOOQF
VROOO
OOOOO
OOOO`
OOONO
ONWaO
OOQ_K
TOFON
HOK\A
OTOOI
OOAOO
OHXOO
OOOOO
OOEOO
OOUFO
OOOGO
SOJOO
OAVOO
OQOOO
OOOOO
OOOHO
OVKOO
OOSOO
OOOOO
OOOOO
OOOOO
OOMGO
OOUOO
OOOON
^OOOO
ONOG^
RIOYO
OOOHG
]OOOO
OCOOA
OOOBO
OOSOJ
O[GOO
OOOMW
OL`OO
OOOOO
O^IOH
OA[OO
OOOOL
UOOXO
O`VOO
OODDO
BOOOH
OOLLO
OOOOO
OZOPO
GROOM
OJOOG
ONGON
OOBOO
ORLOO
OOOAR
OKOOB
OObOO
OOJMO
OOOOO
OOOOO
OOXIO
OOJOO
OOO`O
OOOOO
OOObO
OLOOO
OOOSO
OO^OO
OOOOO
OOO^c
OOOOK
OOOOE
OOOOO
SOOSC
OOUJO
OOOSO
BOKOC
ObOSO
OOOOO
OOOOO
OJOOQ
OOOOO
OOOOC
SOOOO
bOOOO
OOAQO
OOOO^
OBOHO
OOOOO
OOMOR
OOOOO
OOME_
OOGCO
OOOTO
OO[OO
OOBVO
OV[OV
OOaOO
OOOOB
O[OOO
OOHGO
NOOOO
OAOOA
OO`OM
OOJOO
OOOOO
OOOO_
OOJOD
OOWMO
O_OOM
OOMOO
OOOOR
OOE^Q
OOOOO
OOOOO
OODON
OOOOO
OOOOL
OOTOO
OO\O^
OVOOO
OOOOa
OOOQI
OOObX
OAWOL
NKOO

In [94]:
result_dataFrame['phone']

0      OOOOc
1      OOOOO
2      OO^^B
3      OOKOb
4      OOCOH
       ...  
495    OO_RO
496    OOLIO
497    O`OOO
498    OTOO_
499    OOOOO
Name: phone, Length: 500, dtype: object

In [95]:
result_dataFrame.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOc,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,OOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^B,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


In [96]:
result_dataFrame.rename(columns = {'phone':'coded_phone_number'}, inplace = True)

In [97]:
result_dataFrame.head()

,name,coded_phone_number,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOc,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,OOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^B,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
